In [44]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.display import display
from io import StringIO
import csv
import re
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Alignment, Font
import traceback


protoids = ['29401','29352','29463','29249','29244']

def generate_sat_template(protoid):

    pd.set_option('display.max_colwidth', None)

    try:
        with open(f"html_{protoid}.html", "r", encoding="utf-8") as file:
            trigger_history_container = file.read()
    except FileNotFoundError:
        print("The file does not exist!")

    # Parse the div content with BeautifulSoup
    soup = BeautifulSoup(trigger_history_container, 'html5lib')

    # Find the conditions div "box-main"
    box_main = soup.find_all('div', {'class': 'box-main'}) #box_main is a list of div elements, each containing a condition
    conditions = pd.DataFrame()
    actions = pd.DataFrame()
    group_n = []
    condition_n = []

    sat_n = 0
    for element in box_main: # under each sat
    
        if "(Suspended)" in str(element): # Skipping sats with (Suspended) questions
            continue

        sat_n += 1 # defines trigger number
    
        # Extracting condition table
        condition_table = element.find('table', id=lambda x: x and x.startswith('conditionDescriptionTable'))
        block_exclude = element.find_all('div', {'class': 'historySectionLevelsDiv'}) # Finds section names
        block_text = []
        tr = condition_table.find_all('tr')

        for row in tr: # each row is a line of the condition

            # take the condition
            condition_text = row.get_text(strip=True)

            exclude_texts = [tag.get_text(strip=True) for tag in block_exclude if tag]

            for text_exclude in exclude_texts:  # Iterate through the extracted texts
                if text_exclude in condition_text:  # Check if the text exists in condition_text
                    condition_text = condition_text.replace(text_exclude, '')  # Perform the replacement
            
            #take the condition and group number, if any
            input_tag = row.find('input', {'name': 'editremove'})
            #print(input_tag)
            if input_tag:
                value_attr = input_tag.get('value')
                if value_attr:  # Ensure value_attr is not None

                    if '"group":' in value_attr:
                        group_start = value_attr.find('"group":') + len('"group":')
                        group_end = value_attr.find(',', group_start)
                        group_n = value_attr[group_start:group_end].strip()
                    else:
                        group_n = None
        
                    if '"condition":' in value_attr:
                        condition_start = value_attr.find('"condition":') + len('"condition":')
                        condition_end = value_attr.find(',', condition_start) if ',' in value_attr[condition_start:] else value_attr.find('}', condition_start)
                        condition_n = value_attr[condition_start:condition_end].strip()
                    else:
                        condition_n = None

            block_text.append({'sat_n': sat_n, 'group_n': group_n, 'condition_n': condition_n, 'condition_text': condition_text})

        df_condition = pd.DataFrame(block_text, columns=block_text[0])
        conditions = pd.concat([conditions,df_condition],ignore_index=True)
    
        # Extracting action table
        action_table = element.find('table', id=lambda x: x and x.startswith('historyActionTable'))
        if action_table:
            # Convert the second table into a dataframe
            rows = action_table.find_all('tr')
            table_data = []
            for row in rows:
                cols = row.find_all(['th', 'td'])
                cols_text = [col.get_text(strip=True) for col in cols]
                table_data.append(cols_text)

        # Storing the action table in a DataFrame and adding the condition in a column
        df_action = pd.DataFrame(table_data[1:], columns=table_data[0])  # [1:] skips the header row, table_data[0] is the header
        df_action['sat_n'] = sat_n
        actions = pd.concat([actions,df_action],ignore_index=True)

    sat = pd.merge(conditions, actions, on='sat_n', how='left') # Appending all the dataframes for each box_main (sat)
    sat['group_n'] = sat['group_n'].astype(int) + 1
    sat['condition_n'] = sat['condition_n'].astype(int) + 1
    sat['group_n'] = sat['group_n'].apply(lambda x: f"Group {int(x)}")
    sat['condition_n'] = sat['condition_n'].apply(lambda x: f"Condition {int(x)}")
    
    # Renaming columns
    new_column_names = {
        'sat_n': 'sat_n',
        'condition_text': 'cond_question',
        'Action': 'act_action_question',
        'Question': 'act_parameter',
        'Answers': 'act_action_answer',
        'Comment': 'act_action_comment'
    }
    sat = sat.rename(columns=new_column_names)
    sat = sat[['sat_n','group_n','condition_n','cond_question', 'act_parameter','act_action_answer', 'act_action_comment','act_action_question']]
    sat = sat.sort_values(by=['sat_n','group_n','condition_n'], ascending=[True, True, True])

    # Breaking the tables into action and condition to be processed separately
    conditions = sat[['sat_n','group_n','condition_n','cond_question']]
    actions = sat[['sat_n','act_parameter','act_action_answer', 'act_action_comment','act_action_question']]

    # Condition processing

    # Breaking down cond_question

    conditions = conditions.copy()

    # Breaking down cond_question
    conditions.loc[:,'cond_logical_operator'] = conditions['cond_question'].apply(lambda x: re.search(r'(AND|OR)', str(x)).group(0) if re.search(r'(AND|OR)', str(x)) else None) # picks up the words AND or OR, if any
    conditions.loc[:,'cond_question'] = conditions.apply(lambda row: row['cond_question'].replace(row['cond_logical_operator'], '') if row['cond_logical_operator'] else row['cond_question'],axis=1) # removes logical operator from main string, if any
    conditions.loc[:,'cond_operator'] = conditions['cond_question'].apply(lambda x: re.search(r'\b[A-Z][A-Z\s]*\b', str(x)).group(0) if re.search(r'\b[A-Z][A-Z\s]*\b', str(x)) else None) # picks up capitalised strings (IS IN, IS NOT IN...)
    conditions.loc[:, 'cond_parameter'] = conditions.apply(lambda row: (row['cond_question'].split(' [')[0] if isinstance(row['cond_question'], str) and ' [' in row['cond_question'] else (row['cond_question'].split(row['cond_operator'])[0].strip() if row['cond_operator'] and row['cond_operator'] in row['cond_question'] else None)),axis=1)
    conditions['cond_parameter_value'] = conditions['cond_question'].apply(lambda x: x.split(" (")[1].split(")")[0] if " (" in x and ")" in x else None) # takes string between ()

    # Remove Question ID from cond_question
    conditions.loc[:, 'cond_parameter'] = conditions['cond_parameter'].apply(lambda x: x.split(' [')[0] if isinstance(x, str) and ' [' in x else x)

    # Set Parameter_type is always Answer for Condition
    conditions.loc[:,'cond_parameter_type'] = 'Answer'

    # Creating extra columns Type and Name
    conditions['cond_type'] = 'Condition'     

    # Conditionally update rows containing "Group"
    conditions.loc[conditions['cond_question'].str.contains('Group', case=False, na=False),['condition_n','cond_operator', 'cond_parameter', 'parameter_value']] = [None, None, None, None]
    conditions.loc[conditions['cond_question'].str.contains('Group', case=False, na=False), 'cond_parameter_type'] = "Group"
    conditions.loc[~conditions['cond_question'].str.contains('Group', case=False, na=False), 'cond_parameter_type'] = "Answer"

    # Adding empty columns for unioning
    conditions.loc[:,'cond_action'] = None
    conditions.loc[:,'cond_action_option'] = None

    conditions = conditions[['sat_n','group_n','condition_n','cond_type','cond_parameter_type','cond_parameter','cond_operator','cond_parameter_value','cond_action','cond_action_option','cond_logical_operator']]

    # Break down condition answers and placing them in different columns
    answer_split = conditions['cond_parameter_value'].str.split(';', expand=True) # Split by ;
    answer_split.columns = [f'cond_parameter_value_{i+1}' for i in range(answer_split.shape[1])] # assigning each answer to its own column with an identifier
    conditions = pd.concat([conditions,answer_split], axis=1) # Appending new columns

    # Turning the answers into rows
    conditions_unpivot = pd.melt(conditions, 
                       id_vars=['sat_n','group_n','condition_n','cond_type','cond_parameter_type','cond_parameter','cond_operator','cond_action','cond_action_option','cond_logical_operator'], 
                       value_vars=answer_split.columns,
                       var_name='answer', 
                       value_name='parameter_value')

    # Tidying the table and adding empty columns to align with the action table (for the union)
    conditions_final = conditions_unpivot.rename(columns={'parameter_value': 'cond_parameter_value'})
    conditions_final = conditions_final[['sat_n','group_n','condition_n','cond_type','cond_parameter_type','cond_parameter','cond_operator','cond_parameter_value','cond_action','cond_action_option','cond_logical_operator']]
    conditions_final = conditions_final.drop_duplicates()
    conditions_final = conditions_final[conditions_final['cond_parameter_value'].notna()]
    conditions_final = conditions_final.sort_values(by=['sat_n','group_n','condition_n'], ascending=[True, True, True])

    # Action processing - part 1

    # Attachement
    actions = actions.copy()

    # Remove Question ID from act_parameter
    actions.loc[:,'act_parameter'] = actions['act_parameter'].apply(lambda x: x.split(' [')[0])

    # Setting up Types for Answer and Comment, normalising Comment action
    actions.loc[:, 'Parameter Type_1'] = actions['act_action_answer'].apply(lambda x: 'Answer' if x != "" else None)
    actions.loc[:, 'Parameter Type_2'] = actions['act_action_comment'].apply(lambda x: 'Comment' if x != "" else None)
    actions.loc[:,'act_action_comment'] = actions['act_action_comment'].apply(lambda x: 'Change' if x == 'Set Comment(comment: )' else None)

    # Break down act_action_answer (separating the action-answer combinations)
    action_split = actions['act_action_answer'].str.split(')', expand=True)
    action_split.columns = [f'act_action_answer_{i+1}' for i in range(action_split.shape[1])] # assigning each action-answer to its own column with an identifier
    actions = pd.concat([actions, action_split], axis=1)

    # Separate action-answer into respective columns with identifier
    action_split2 = pd.DataFrame()
    for i in range(action_split.shape[1]): #action_split.shape[1] is the number of action-answer combinations
        column_name = f'act_action_answer_{i+1}'
        action_split2[[f'act_action_{i+1}', f'act_parameter_value_{i+1}']] = actions[column_name].apply(
            lambda x: pd.Series(str(x).split('(', 1)) if isinstance(x, str) and '(' in str(x) else pd.Series([None, None]))

    actions = actions[['sat_n','act_parameter', 'act_action_comment','act_action_question']]
    actions = pd.concat([actions, action_split2], axis=1)

    # Action processing - part 2

    # Unpivotting the columns to have actions and answers as rows pertaining to the question

    # Unpivot parameter_type=comment actions
    unpivoted_1 = pd.melt(actions, id_vars=['sat_n', 'act_parameter'], value_vars=['act_action_comment'], var_name='act_parameter_type', value_name='act_action')
    unpivoted_1 = unpivoted_1[unpivoted_1['act_action'].notna()]
    unpivoted_1.loc[:,"act_parameter_type"] = "Comment"

    # Unpivot parameter_type=question actions
    unpivoted_2 = pd.melt(actions, id_vars=['sat_n', 'act_parameter'], value_vars=['act_action_question'], var_name='act_parameter_type', value_name='act_action')
    unpivoted_2 = unpivoted_2[unpivoted_2['act_action'].notna()]
    unpivoted_2.loc[:,"act_parameter_type"] = "Question"

    # Unioning them and setting up empty column to align with the parameter_type=answer table below
    question_answer = pd.concat([unpivoted_1,unpivoted_2], ignore_index=True)
    question_answer.loc[:,"act_parameter_value"] = None
    question_answer = question_answer[['sat_n','act_parameter_type','act_parameter', "act_parameter_value",'act_action']]

    question_answer = question_answer.drop_duplicates()

    # Actions processing - part 3

    # Manipulating the parameter_type=answer actions and values to be in the template shape

    # Unpivotting the actions, defining identifier column for the action-answer combination (action_n), setting up id for left join
    unpivoted_dfs = []
    action_columns = [col for col in actions.columns if col.startswith('act_action_') and col.split('_')[-1].isdigit()]
    for act_action in action_columns:
        unpivoted_n = pd.melt(actions, id_vars=['sat_n', 'act_parameter'], value_vars=[act_action], var_name='act_parameter_type', value_name='value')
        unpivoted_dfs.append(unpivoted_n)

    action_unpivoted= pd.concat(unpivoted_dfs, ignore_index=True)
    action_unpivoted = action_unpivoted[action_unpivoted['value'].notna()]
    action_unpivoted.loc[:, 'action_n'] = action_unpivoted['act_parameter_type'].apply(lambda x: x.split('_')[-1] if x.split('_')[-1].isdigit() else None)
    action_unpivoted.loc[:, 'act_parameter_type'] = action_unpivoted['act_parameter_type'].apply(lambda x: "act_action" if "action" in x else None)
    action_unpivoted.loc[:, 'id'] = action_unpivoted['sat_n'].astype(str) + action_unpivoted['act_parameter'].astype(str) + action_unpivoted['action_n'].astype(str)

    # Unpivotting the answers, defining identifier column for the action-answer combination (action_n), setting up id for left join
    unpivoted_dfs = []
    param_columns = [col for col in actions.columns if col.startswith('act_parameter_value_')]
    for act_param in param_columns:
        unpivoted_n = pd.melt(actions, id_vars=['sat_n', 'act_parameter'], value_vars=[act_param], var_name='act_parameter_type', value_name='value')
        unpivoted_dfs.append(unpivoted_n)

    param_unpivoted= pd.concat(unpivoted_dfs, ignore_index=True)
    param_unpivoted = param_unpivoted[param_unpivoted['value'].notna()]
    param_unpivoted.loc[:, 'action_n'] = param_unpivoted['act_parameter_type'].apply(lambda x: x.split('_')[-1] if x.split('_')[-1].isdigit() else None)
    param_unpivoted.loc[:, 'act_parameter_type'] = param_unpivoted['act_parameter_type'].apply(lambda x: "act_parameter_value" if "parameter_value" in x else None)
    param_unpivoted.loc[:, 'id'] = param_unpivoted['sat_n'].astype(str) + param_unpivoted['act_parameter'].astype(str) + param_unpivoted['action_n'].astype(str)

    # Left join attaching the answers to the actions (there'll always be an action, but not always an answer)
    actions_param = pd.merge(action_unpivoted, param_unpivoted, how='left', on='id',suffixes=('_action', '_parameter'))

    # Tidying up the table, setting up type
    actions_param = actions_param.sort_values(by=['sat_n_action','act_parameter_action','action_n_action'], ascending=[True, True, True])
    actions_param.loc[:,"act_parameter_type"] = "Answer"
    actions_param = actions_param[['sat_n_action','act_parameter_type','act_parameter_action','value_parameter','value_action']]

    actions_param = actions_param.rename(columns={
        'sat_n_action': 'sat_n',
        'act_parameter_type': 'act_parameter_type',
        'act_parameter_action': 'act_parameter',
        'value_parameter': 'act_parameter_value',
        'value_action': 'act_action'
    })

    actions_param = actions_param.drop_duplicates()

    # Actions processing - part 4

    # Unioning the parameter_types together and adding empty columns to align with Conditions table for unioning
    actions_final = pd.concat([question_answer, actions_param], ignore_index=True)
    actions_final.loc[:,"act_type"] = "Action"
    actions_final.loc[:,"act_group_n"] = None
    actions_final.loc[:,"act_condition_n"] = None
    actions_final.loc[:,"act_operator"] = None
    actions_final.loc[:,"act_action_option"] = None
    actions_final.loc[:,"act_logical_operator"] = None

    actions_final = actions_final.sort_values(by=['sat_n','act_parameter_type','act_parameter','act_action'], ascending=[True, True, True,True])
    actions_final = actions_final[['sat_n','act_group_n','act_condition_n','act_type','act_parameter_type','act_parameter','act_operator','act_parameter_value','act_action','act_action_option','act_logical_operator']]

    actions_final = actions_final.drop_duplicates()

    # Merging conditions and actions

    # renaming conditions columns
    conditions_final = conditions_final.rename(columns={
        'sat_n': 'sat_n',
        'cond_type': 'type',
        'cond_parameter_type': 'parameter_type',
        'cond_parameter': 'parameter',
        'cond_operator': 'operator',
        'cond_parameter_value': 'parameter_value',
        'cond_action': 'action',
        'cond_action_option': 'action_option',
        'cond_logical_operator': 'logical_operator',
    })

    # renaming action columns
    actions_final = actions_final.rename(columns={
        'sat_n': 'sat_n',
        'act_type': 'type',
        'act_parameter_type': 'parameter_type',
        'act_parameter': 'parameter',
        'act_operator': 'operator',
        'act_parameter_value': 'parameter_value',
        'act_action' : 'action',
        'act_action_option': 'action_option',
        'act_logical_operator': 'logical_operator',
    })

    sat_combined = pd.concat([conditions_final, actions_final], ignore_index=True)
    sat_combined = sat_combined.sort_values(by=['sat_n','type','parameter'], ascending=[True, False, True])

    # Equivalencies of operators and actions


    # Operator equivalence
    sat_combined['operator'] = sat_combined['operator'].apply(lambda x: 'in' if x == 'IS IN' else '!in' if x == 'IS NOT IN' else None)

    # Action equivalence

    #add attachment

    action_mapping = {
        'show': 'Show',
        'hide': 'Hide',
        'Change': 'Change',
        'Set Answer': 'Set',
        'Clear Answer': 'Clear',
        'Disable Answer': 'Disable',
        'Enable Answer': 'Enable',
        'Show Answer': 'Show',
        'Hide Answer': 'Hide',
    }
    sat_combined['action'] = sat_combined['action'].apply(lambda x: action_mapping.get(x, None))

    # Formatting the final table to the template shape

    # Function to add "Rule" and "Group" rows
    def add_null_rows(group):
        rule_row = {'sat_n': group['sat_n'].iloc[0], 'type': 'Rule'}
        group_row = {'sat_n': group['sat_n'].iloc[0], 'type': 'Group'}
        return pd.concat([pd.DataFrame([rule_row, group_row]), group])

    # Apply the function and grouping by sat_n
    sat_final = sat_combined.groupby('sat_n')[sat_combined.columns.tolist()].apply(add_null_rows).reset_index(drop=True)

    # Add Name column after type
    sat_final.loc[:,'name'] = sat_final.apply(lambda row: sat_final.loc[row.name + 1, 'group_n'] if row['type'] == 'Group' and row.name + 1 < len(sat_final) else (row['condition_n'] if row['type'] == 'Condition' else None), axis=1)

    sat_final = sat_final[['sat_n','type','name','parameter_type','parameter','operator','parameter_value','action','action_option','logical_operator']]    

    # Looking up question codes and replacing

    # Load your CSV file into a DataFrame
    datalist = pd.read_excel(f'datalist_{protoid}.xlsx', sheet_name='DataLists')

    # Create a new column to store the matching values
    sat_final['parameter'] = sat_final['parameter'].apply(lambda param: next((row for row in datalist.iloc[:, 0] if str(param) in str(row)),param))

    # Creating excel file

    # Aligning column names
    sat_final = sat_final[['type','name','parameter_type','parameter','operator','parameter_value','action','action_option','logical_operator']]
    sat_final = sat_final.rename(columns={
        'name': 'Name',
        'parameter_type': 'Parameter Type',
        'parameter': 'Parameter',
        'operator': 'Statement Operator',
        'parameter_value': 'Parameter Value',
        'action': 'Action',
        'action_option': 'Action Option',
        'logical_operator': 'Logical Operator'
    })
    
    
    # Create a new workbook and select the active sheet
    wb = Workbook()
    ws = wb.active

    # Write the dataframe into the sheet, including headers
    for col_idx, column in enumerate(sat_final.columns, start=1):
        cell = ws.cell(row=1, column=col_idx, value=column) 
        cell.font = Font(bold=True)  # Make headers bold
        cell.alignment = Alignment(horizontal="center", vertical="center")  # Center the text

    for row_idx, row in sat_final.iterrows():
        for col_idx, value in enumerate(row, start=1):
            cell = ws.cell(row=row_idx + 2, column=col_idx, value=value)  # Data rows
            cell.alignment = Alignment(horizontal="center", vertical="center")

    # Define the fill colors
    colors = {
        'Rule': 'D88A55',
        'Group': '00B09B',
        'Condition': '9DD3C9',
        'Action': 'D9D9D9'
    }

    # Apply conditional formatting row by row
    for row_idx, row in sat_final.iterrows():
        fill_color = colors.get(row['type'], None)
        if fill_color:
            for col_idx in range(1, len(sat_final.columns) + 1):  # Format all cells in the row
                ws.cell(row=row_idx + 2, column=col_idx).fill = PatternFill(
                    start_color=fill_color,
                    end_color=fill_color,
                    fill_type="solid"
                )

    for column_cells in ws.columns:
        ws.column_dimensions[column_cells[0].column_letter].width = 20

    # Save the workbook
    wb.save(f'template_{protoid}.xlsx')

for protoid in protoids:
    try:
        generate_sat_template(protoid)
        print(f"Processed and saved template files for: {protoid}")
    except Exception as e:
        tb = traceback.extract_tb(e.__traceback__)
        line_number = tb[-1].lineno 
        print(f"Error processing {protoid}: {e}")
        print(f"Error occurred on line number: {line_number}")
        print(f"Failed operation: {tb[-1]}")


Processed and saved template files for: 29401
Processed and saved template files for: 29352
Processed and saved template files for: 29463
Processed and saved template files for: 29249
Processed and saved template files for: 29244


In [49]:
!pip freeze > requirements.txt